In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [0]:
WORKING_DIR = "/content/drive/My Drive/pix2pix/jmamoon/hairstyle_filtered"

INPUT_DIR_NAME = "/input"
OUTPUT_DIR_NAME = "/output"
CHECKPOINTS_DIR_NAME = "/checkpoints"

In [0]:
INPUT_DIR = WORKING_DIR + INPUT_DIR_NAME
OUTPUT_DIR = WORKING_DIR + OUTPUT_DIR_NAME
CHECKPOINTS_DIR = WORKING_DIR + CHECKPOINTS_DIR_NAME

In [0]:
def read_images(directory):
  images = !ls -1 "{directory}"
  return images

In [0]:
all_inputs = read_images(INPUT_DIR)

In [0]:
def slice_images(images, quantity):
  return np.copy(images[:quantity])

def shuffle_images(images, seed=None):
  if seed is not None:
    np.random.seed(seed)
  np.random.shuffle(images)

def calculate_training_size(total, percent):
  return round(total * percent)

def slice_training_images(images, percent):
  total = len(images)
  size = calculate_training_size(total, percent)
  return images[:size]

def slice_test_images(images, percent):
  total = len(images)
  size = calculate_training_size(total, percent)
  return images[size:total]

In [0]:
DATA_SIZE = 500
TRAINING_PERCENT = 0.8
SHUFFLE_SEED = 37

random_inputs = slice_images(all_inputs, DATA_SIZE)
shuffle_images(random_inputs, SHUFFLE_SEED)

training_inputs = slice_training_images(random_inputs, TRAINING_PERCENT)
testing_inputs = slice_test_images(random_inputs, TRAINING_PERCENT)

In [0]:
import random

In [0]:
def resize_image(image, width, height):
  return tf.image.resize(image, [width, height])

def resize_image_pair(input_image, output_image, height, width):
  input_image = resize_image(input_image, width, height)
  output_image = resize_image(output_image, width, height)
  return input_image, output_image

def normalize_image(image):
  return (image / 127.5) - 1

def normalize_image_pair(input_image, output_image):
  input_image = normalize_image(input_image)
  output_image = normalize_image(output_image)
  return input_image, output_image

@tf.function
def random_jitter_image_pair(input_image, output_image, width, height, offset):
  new_width = width + offset
  new_height = height + offset
  input_image = resize_image(input_image, new_width, new_height)
  output_image = resize_image(output_image, new_width, new_height)
  
  stacked_image = tf.stack([input_image, output_image], axis = 0)
  cropped_stacked_image = tf.image.random_crop(stacked_image, size = [2, width, height, 3])
  input_image = cropped_stacked_image[0]
  output_image = cropped_stacked_image[1]
  
  if (random.randint(0, 100) / 100) > 0.5:
    input_image = tf.image.flip_left_right(input_image)
    output_image = tf.image.flip_left_right(output_image)
  
  return input_image, output_image

def load_image_from_file_system(directory, filename):
  return tf.cast(tf.image.decode_jpeg(tf.io.read_file(directory + "/" + filename)), tf.float32)[..., :3]

def load_image_pair(filename, width, height, random_jitter_offset = 0):
  input_image = load_image_from_file_system(INPUT_DIR, filename)
  output_image = load_image_from_file_system(OUTPUT_DIR, filename)
  
  input_image, output_image = resize_image_pair(input_image, output_image, height, width)
  
  if random_jitter_offset > 0:
    input_image, output_image = random_jitter_image_pair(input_image, output_image, width, height, random_jitter_offset)
  
  input_image, output_image = normalize_image_pair(input_image, output_image)
  
  return input_image, output_image

def load_image_pair_for_training(filename, width, height):
  return load_image_pair(filename, width, height, 30)

def load_image_pair_for_testing(filename, width, height):
  return load_image_pair(filename, width, height)

In [12]:
IMAGE_WIDTH = 256
IMAGE_HEIGHT = 256

dataset_for_training = tf.data.Dataset.from_tensor_slices(training_inputs)
dataset_for_training = dataset_for_training.map(lambda training_input: load_image_pair_for_training(training_input, IMAGE_WIDTH, IMAGE_HEIGHT), num_parallel_calls = tf.data.experimental.AUTOTUNE)
dataset_for_training = dataset_for_training.batch(1)

dataset_for_testing = tf.data.Dataset.from_tensor_slices(testing_inputs)
dataset_for_testing = dataset_for_testing.map(lambda testing_input: load_image_pair_for_testing(testing_input, IMAGE_WIDTH, IMAGE_HEIGHT), num_parallel_calls = tf.data.experimental.AUTOTUNE)
dataset_for_testing = dataset_for_testing.batch(1)

In [0]:
from tensorflow.keras import *
from tensorflow.keras.layers import *

def downsample(filters, apply_batch_normalization = True):
  layer = Sequential()
  
  initializer = tf.random_normal_initializer(0, 0.02)
  
  layer.add(Conv2D(filters,
                   kernel_size = 4,
                   strides = 2,
                   padding = "same",
                   kernel_initializer = initializer,
                   use_bias = not apply_batch_normalization))
  
  if apply_batch_normalization:
    layer.add(BatchNormalization())
  
  layer.add(LeakyReLU())
  
  return layer

def upsample(filters, apply_dropout = False):
  layer = Sequential()
  
  initializer = tf.random_normal_initializer(0, 0.02)
  
  layer.add(Conv2DTranspose(filters,
                            kernel_size = 4,
                            strides = 2,
                            padding = "same",
                            kernel_initializer = initializer,
                            use_bias = False))
  
  layer.add(BatchNormalization())
  
  if apply_dropout:
    layer.add(Dropout(0.5))
  
  layer.add(ReLU())
  
  return layer

In [0]:
def Generator():
  inputs = tf.keras.layers.Input(shape = [None, None, 3])
  
  down_stack = [
      downsample(64, apply_batch_normalization = False),
      downsample(128),
      downsample(256),
      downsample(512),
      downsample(512),
      downsample(512),
      downsample(512),
      downsample(512),
  ]
  
  up_stack = [
      upsample(512, apply_dropout = True),
      upsample(512, apply_dropout = True),
      upsample(512, apply_dropout = True),
      upsample(512),
      upsample(256),
      upsample(128),
      upsample(64),
  ]
  
  initializer = tf.random_normal_initializer(0, 0.02)
  
  last = Conv2DTranspose(filters = 3,
                         kernel_size = 4,
                         strides = 2,
                         padding = "same",
                         kernel_initializer = initializer,
                         activation = "tanh")
  
  x = inputs
  s = []
  
  concat = Concatenate()
  
  for down in down_stack:
    x = down(x)
    s.append(x)
  
  s = reversed(s[:-1])
  
  for up, sk in zip(up_stack, s):
    x = up(x)
    x = concat([x, sk])
  
  last = last(x)
  
  return Model(inputs = inputs, outputs = last)

ini_img = None
for a in dataset_for_training.take(1):
  ini_img = a


generator = Generator()

In [0]:
def Discriminator():
  ini = Input(shape = [None, None, 3], name = "input_img")
  gen = Input(shape = [None, None, 3], name = "gener_img")
  
  con = concatenate([ini, gen])
  
  initializer = tf.random_normal_initializer(0, 0.02)
  
  down1 = downsample(64, apply_batch_normalization = False)(con)
  down2 = downsample(128)(down1)
  down3 = downsample(256)(down2)
  down4 = downsample(512)(down3)
  
  last = tf.keras.layers.Conv2D(filters = 1,
                               kernel_size = 4,
                               strides = 1,
                               kernel_initializer = initializer,
                               padding = "same")(down4)
  
  return tf.keras.Model(inputs = [ini, gen], outputs = last)

discriminator = Discriminator()

In [0]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits = True)

In [0]:
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)
  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)
  
  total_disc_loss = real_loss + generated_loss
  
  return total_disc_loss

In [0]:
LAMBDA = 100

def generator_loss(disc_generated_output, gen_output, target):
  gen_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))
  
  total_gen_loss = gen_loss + (LAMBDA * l1_loss)
  
  return total_gen_loss

In [0]:
import os

generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

checkpoint_prefix = os.path.join(CHECKPOINTS_DIR, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer = generator_optimizer,
                                discriminator_optimizer = discriminator_optimizer,
                                generator = generator,
                                discriminator = discriminator)

#checkpoint.restore(tf.train.latest_checkpoint(CHECKPOINTS_DIR_NAME)).assert_consumed() 

In [0]:
def generate_images(model, test_input, tar, save_filename = False, display_imgs = True, training = True):
  prediction = model(test_input, training)
  
  if save_filename:
    tf.keras.preprocessing.image.save_img(WORKING_DIR + save_filename + ".jpg", prediction[0, ...])
  
  plt.figure(figsize = (10, 10))
  
  display_list = [test_input[0], tar[0], prediction[0]]
  title = ["input image", "ground truth", "predicted image"]
  
  if display_imgs:
    for i in range(3):
      plt.subplot(1, 3, i + 1)
      plt.title(title[i])
      plt.imshow(display_list[i] * 0.5 + 0.5)
      plt.axis("off")
  
  plt.show()

In [0]:
@tf.function
def train_step(input_image, target):
  
  with tf.GradientTape() as gen_tape, tf.GradientTape() as discr_tape:
  
    output_image = generator(input_image, training = True)

    output_gen_discr = discriminator([output_image, input_image], training = True)

    output_trg_discr = discriminator([target, input_image], training = True)

    disc_loss = discriminator_loss(output_trg_discr, output_gen_discr)

    gen_loss = generator_loss(output_gen_discr, output_image, target)

    generator_grads = gen_tape.gradient(gen_loss, generator.trainable_variables)

    discriminator_grads = discr_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(generator_grads, generator.trainable_variables))

    discriminator_optimizer.apply_gradients(zip(discriminator_grads, discriminator.trainable_variables))

In [0]:
from IPython.display import clear_output

def train(dataset, epochs, current_epoch = 0):
  for epoch in range(epochs)[current_epoch:]:
    imgi = 0
    for input_image, target in dataset:
      print("epoch " + str(epoch) + " - train: " + str(imgi) + "/" + str(len(training_inputs)))
      imgi += 1
      train_step(input_image, target)
    
      clear_output(wait = True)
    
    if (epoch + 1) %25 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

      imgi = 0
      for inp, tar in dataset_for_testing.take(5):
        generate_images(generator, inp, tar, "/generated_training/" + str(imgi) + "_" + str(epoch), display_imgs = True)
        imgi += 1

In [25]:
train(dataset_for_training, 1000)

epoch 339 - train: 176/277


KeyboardInterrupt: ignored

In [26]:
imgi = 0
for inp, tar in dataset_for_testing.take(20):
  generate_images(generator, inp, tar, "/generated_testing/testing_" + str(imgi), display_imgs = True)
  imgi += 1
  

Output hidden; open in https://colab.research.google.com to view.

In [0]:
asdasd = 14
